In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cross_validation import StratifiedKFold

% matplotlib inline

In [2]:
churn_data = pd.read_csv('Churn_Modelling.csv')

In [3]:
churn_data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
churn_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


### Data preprocessing

Obtain the independent and dependent variable

In [5]:
X = churn_data.iloc[:, 3:-1]
y = churn_data.iloc[:, -1]

In [6]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [7]:
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

Encode the categorical data

In [8]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

Label_X_1 = LabelEncoder()
X.iloc[:,1] =  Label_X_1.fit_transform(X.iloc[:,1])
Label_X_2 = LabelEncoder()
X.iloc[:,2] =  Label_X_2.fit_transform(X.iloc[:,2])

In [9]:
# Using one hot encoding from Pandas
X1 = X
one_hot_geo = pd.get_dummies(X1['Geography'], prefix='Geography', drop_first=True)
X1 = X1.join(one_hot_geo)

In [10]:
X1.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_1,Geography_2
0,619,0,0,42,2,0.00,1,1,1,101348.88,0,0
1,608,2,0,41,1,83807.86,1,0,1,112542.58,0,1
2,502,0,0,42,8,159660.80,3,1,0,113931.57,0,0
3,699,0,0,39,1,0.00,2,0,0,93826.63,0,0
4,850,2,0,43,2,125510.82,1,1,1,79084.10,0,1


In [166]:
# Using one hot encoding from sklearn
one_hot_geo_2 = OneHotEncoder(categorical_features=[1])
X_2 = X
X_2 = one_hot_geo_2.fit_transform(X).toarray()

In [167]:
# Remove one column of the dummy variables
X1.drop(labels='Geography_2', axis=1, inplace=True)

In [168]:
X1.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_0,Geography_1
0,619,0,42,2,0.00,1,1,1,101348.88,1,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,0
3,699,0,39,1,0.00,2,0,0,93826.63,1,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0


### Prepare training and testing set for the model training

In [11]:
y.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

Given that the dependent variable is not uiform, we will use the stratified split

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.1, stratify=y, random_state=0)

In [13]:
X_test.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_1,Geography_2
2883,559,0,0,38,8,95139.41,1,1,1,86575.46,0,0
815,745,0,1,25,5,157993.15,2,1,0,146041.45,0,0
5395,589,1,1,48,5,126111.61,1,0,1,133961.19,1,0
6400,676,2,0,30,5,0.00,2,0,0,179066.58,0,1
8529,603,0,1,45,9,0.00,1,0,0,148516.79,0,0


Feature scaling, critical for training the deep neural network, even for 0/1 categorical variables

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Build Artificial Neural Network

In [15]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


Initialize ANN: Define sequence of layers or define graph, here we will use the first way

In [20]:
# Initialize the ANN
classifier = Sequential()

# Add input layer and first hidden layer
classifier.add(Dense(activation="relu", units=6, kernel_initializer="uniform", input_dim=11))
classifier.add(Dense(activation="relu", units=6, kernel_initializer="uniform"))

# Add output layer
classifier.add(Dense(activation="sigmoid", units=1, kernel_initializer="uniform"))

classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [21]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
classifier.fit(X_train, y_train, batch_size = 32, nb_epoch =100, validation_split=0.1)
y_pred = classifier.predict(X_test)>0.5
from sklearn.metrics import accuracy_score
print(accuracy_score(np.array(y_test), y_pred))
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm.view()

/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 8100 samples, validate on 900 samples
Epoch 1/100
8100/8100 [==============================] - 1s 66us/step - loss: 0.5781 - acc: 0.7937 - val_loss: 0.4317 - val_acc: 0.8144
Epoch 2/100
8100/8100 [==============================] - 0s 31us/step - loss: 0.4403 - acc: 0.7943 - val_loss: 0.4131 - val_acc: 0.8144

array([[774,  22],
       [151,  53]])

### Perform cross validation using keras wrapping for sklearn

In [25]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.layers import Dropout

In [29]:
def build_ann(hidden_u, layer_num, opt='adam', rate=0.1):
    classifier = Sequential()
    classifier.add(Dense(units=hidden_u, activation='relu', kernel_initializer='glorot_uniform', input_dim=12))
    classifier.add(Dropout(rate=rate))
    for i in range(layer_num-1):
        classifier.add(Dense(units=hidden_u, activation='relu', kernel_initializer='glorot_uniform'))
        classifier.add(Dropout(rate=rate))
    classifier.add(Dense(units=1, activation='sigmoid', kernel_initializer='glorot_uniform'))
    classifier.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

In [30]:
params = {
    'hidden_u': 6,
    'layer_num': 4,
}
classifier = KerasClassifier(build_fn=build_ann, batch_size=32, epochs=100, **params)

In [31]:
stratified_cv = StratifiedKFold(y=y_train, n_folds=10)
accuracies_drop = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=stratified_cv)

Epoch 1/100
8099/8099 [==============================] - 1s 102us/step - loss: 0.5945 - acc: 0.7611
Epoch 2/100
8099/8099 [==============================] - 0s 35us/step - loss: 0.5198 - acc: 0.7958

In [34]:
accuracies_drop.mean(),accuracies_drop.std()

(0.85077395940495015, 0.013563171516400177)

### Hyperparameter tuning

In [35]:
from sklearn.model_selection import GridSearchCV

In [40]:
# Create dictionary of hyperparameters that we want to optimize
parameters ={
    'epochs': [50, 100, 200],
    'hidden_u':[6, 12, 24],
    'layer_num': [3, 5, 7],
    'opt': ['Adam', 'rmsprop'],
    'batch_size': [32, 64]
}

In [41]:
# Create grid search object
classifier = KerasClassifier(build_fn=build_ann, verbose=0)
grid_search = GridSearchCV(estimator=classifier, 
                          param_grid=parameters,
                          scoring = 'accuracy',
                          cv= stratified_cv)

In [ ]:
# Fit grid search to the training set
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_.mean()

In [174]:
grid_search.best_score_

0.86344444444444446

In [175]:
best_parameters

{'epochs': 100, 'hidden_u': 24, 'layer_num': 2, 'opt': 'rmsprop'}

In [93]:
# Create classifier with the best parameter
classifier_best = KerasClassifier(build_fn=build_ann_drop, batch_size=32, epochs=200)
# Retrain with all the training data
accuracies_cv = cross_val_score(estimator=classifier_best, X=X_train, y=y_train, cv=10)

Epoch 1/50
8100/8100 [==============================] - 9s 1ms/step - loss: 0.5937 - acc: 0.7944

In [94]:
accuracies_cv

array([ 0.83888889,  0.82      ,  0.83888889,  0.85333333,  0.82444444,
        0.82111111,  0.83222222,  0.82444444,  0.82666667,  0.85333333])

In [176]:
classifier_best_ori = build_ann_drop2(hidden_u=24, layer_num=2, opt='rmsprop')

In [177]:
classifier_best_ori.fit(X_train, y_train, batch_size=32, epochs=100, verbose=0)

In [178]:
y_predict = classifier_best_ori.predict(X_test) > 0.5

In [179]:
from sklearn.metrics import accuracy_score

In [180]:
accuracy_score(y_test, y_predict)

0.85799999999999998